In [ ]:
import pyvista as pv
from pyvista.trame.ui import plotter_ui
from pyvista.plotting.colors import hexcolors
from trame.app import get_server
from trame.ui.vuetify3 import SinglePageWithDrawerLayout
from trame.widgets import vuetify3
from vtkmodules.vtkFiltersSources import vtkConeSource

pv.OFF_SCREEN = True

server = get_server()
state, ctrl = server.state, server.controller

source = vtkConeSource()

pl = pv.Plotter()
actor = pl.add_mesh(source, color='seagreen')

@state.change("resolution")
def update_contour(resolution, **kwargs):
    source.SetResolution(int(resolution))
    ctrl.view_update()

@state.change("color")
def color(color="seagreen", **kwargs):
    actor.prop.color = color
    ctrl.view_update()

with SinglePageWithDrawerLayout(server) as layout:
    with layout.toolbar:
        vuetify3.VSpacer()
        vuetify3.VProgressLinear(
            indeterminate=True,
            absolute=True,
            bottom=True,
            active=("trame__busy",),
        )
        vuetify3.VSelect(
            label="Color",
            v_model=("color", "seagreen"),
            items=("array_list", list(hexcolors.keys())),
            hide_details=True,
            density="compact",
            outlined=True,
            classes="pt-1 ml-2",
            style="max-width: 250px",
        )
    
    with layout.drawer as drawer:
        drawer.width = 325
        vuetify3.VDivider(classes="mb-2")
        vuetify3.VSlider(
            v_model=("resolution", 15),
            min=5,
            max=30,
            hide_details=True,
            density="compact",
            style="max-width: 300px",
            change=ctrl.view_update,
        )

    with layout.content:
        with vuetify3.VContainer(
            fluid=True,
            classes="pa-0 fill-height",
        ):
            # Use PyVista UI template for Plotters
            view = plotter_ui(pl, mode='client')
            ctrl.view_update = view.update

In [ ]:
await layout.ready
layout